In [1]:
import re
import apache_beam as beam
from apache_beam.options.pipeline_options import PipelineOptions

from apache_beam.options.pipeline_options import GoogleCloudOptions
from apache_beam.options.pipeline_options import SetupOptions
from apache_beam.options.pipeline_options import StandardOptions


# bucket_name = 'bu-ds561-jawicamp'
# project_id = 'jacks-project-398813'

def parse_file(file):
    output = []
    filename, content = file
    links = re.findall(r'<a\s+[^>]*\bHREF=["\']([^"\']+\.html)["\'][^>]*>', content)
    
    for link in links:
        output.append((link, filename))
    return output

options = PipelineOptions()
pipeline = beam.Pipeline(options=options)

links = (
        pipeline 
        | 'ReadFiles' >> beam.io.ReadFromTextWithFilename('gs://bu-ds561-jawicamp/*.html')
        | 'ParseFiles' >> beam.FlatMap(parse_file)
        )
# links | 'PrintLinks' >> beam.Map(lambda x: print(f"Processing: {x}"))

outgoing_counts = (
    links 
    | 'ExtractOutgoingLinks' >> beam.Map(lambda link_tuple: link_tuple[0])
    | 'CountOutgoingLinks' >> beam.Map(lambda link: (link, 1))
    | 'GroupAndSumOutgoing' >> beam.CombinePerKey(sum)
)
# outgoing_counts | 'LogOutgoingCounting' >> beam.Map(lambda x: print(f"Counting outgoing link: {x}"))

incoming_counts = (
    links 
    | 'ExtractIncomingLinks' >> beam.Map(lambda link_tuple: (link_tuple[0], link_tuple[1]))
    | 'MapIncomingLinks' >> beam.Map(lambda link_tuple: (link_tuple[0], link_tuple[1]))
    | 'GroupIncomingLinks' >> beam.GroupByKey()
    | 'CountUniqueIncomingLinks' >> beam.Map(lambda x: (x[0], len(set(x[1]))))
)

# results
top_incoming = incoming_counts | 'TopIncoming' >> beam.combiners.Top.Of(5, key=lambda x: x[1])
top_incoming | 'PrintTopIncoming' >> beam.Map(lambda x: print(f"Top Files (Incoming): {x}"))

top_outgoing = outgoing_counts | 'TopOutgoing' >> beam.combiners.Top.Of(5, key=lambda x: x[1])
top_outgoing | 'PrintTopOutgoing' >> beam.Map(lambda x: print(f"Top Files (Outgoing): {x}"))

result = pipeline.run()
result.wait_until_finish()

usage: ipykernel_launcher.py [-h] [--dataflow_endpoint DATAFLOW_ENDPOINT]
                             [--project PROJECT] [--job_name JOB_NAME]
                             [--staging_location STAGING_LOCATION]
                             [--temp_location TEMP_LOCATION] [--region REGION]
                             [--service_account_email SERVICE_ACCOUNT_EMAIL]
                             [--no_auth]
                             [--template_location TEMPLATE_LOCATION]
                             [--label LABELS] [--update]
                             [--transform_name_mapping TRANSFORM_NAME_MAPPING]
                             [--enable_streaming_engine]
                             [--dataflow_kms_key DATAFLOW_KMS_KEY]
                             [--create_from_snapshot CREATE_FROM_SNAPSHOT]
                             [--flexrs_goal {COST_OPTIMIZED,SPEED_OPTIMIZED}]
                             [--dataflow_service_option DATAFLOW_SERVICE_OPTIONS]
                           

AttributeError: 'tuple' object has no attribute 'tb_frame'